In [ ]:
# This cell makes sure modules are auto-loaded when you change external python files
%load_ext autoreload
%autoreload 2

In [ ]:
import torch
from torch import optim

from datasets import load_metric

from preprocessing import (
    get_sst2_dataset, 
    get_nli_dataset, 
    get_ner_dataset, 
    get_squad_dataset
)

from transformers import (
    AutoModelForQuestionAnswering, 
    AutoModelForSequenceClassification, 
    AutoModelForTokenClassification, 
    AutoTokenizer, 
    TrainingArguments, 
    Trainer, 
    DataCollatorForTokenClassification, 
    DataCollatorWithPadding,
    default_data_collator
)

In [ ]:
seed = 42
torch.manual_seed(seed)

In [ ]:
if torch.cuda.is_available():
  print("Good to go!")
else:
  print("Please connect to a GPU to run this notebook.")

In [ ]:
def retrieve_model(fine_tunning_task, model_name="roberta-base"):
    if fine_tunning_task == "sa":
        tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
        data_collator = DataCollatorWithPadding(tokenizer)
        dataset, encoded_dataset = get_sst2_dataset(tokenizer)
        model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
    elif fine_tunning_task == "nli":
        tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
        data_collator = DataCollatorWithPadding(tokenizer)
        dataset, encoded_dataset = get_nli_dataset(tokenizer)
        model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)
    elif fine_tunning_task == "ner":
        tokenizer = AutoTokenizer.from_pretrained(model_name, add_prefix_space=True)
        data_collator = DataCollatorForTokenClassification(tokenizer)
        dataset, encoded_dataset = get_ner_dataset(tokenizer)
        label_list = dataset["train"].features["ner_tags"].feature.names
        model = AutoModelForTokenClassification.from_pretrained(model_name, num_labels=len(label_list))
    elif fine_tunning_task == "qa":
        tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
        data_collator = default_data_collator
        dataset, encoded_dataset = get_squad_dataset(tokenizer)
        model = AutoModelForQuestionAnswering.from_pretrained(model_name)
    else:
        raise ValueError("Invalid task")

    return dataset, encoded_dataset, data_collator, tokenizer, model

In [ ]:
# load_metric("glue", "sst2")

In [ ]:
model_name = "roberta-base"
task = "qa"
dataset, encoded_dataset, data_collator, tokenizer, model = retrieve_model(task, model_name)

In [ ]:
training_args = TrainingArguments(
    output_dir="{}-finetuned-{}".format(model_name, task),
    learning_rate=2e-5,
    num_train_epochs=5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    # weight_decay=0.01,
    evaluation_strategy = "epoch", # "no" to avoid evaluation
    save_strategy = "epoch", # "no" to avoid saving
    logging_steps=500,
    report_to="tensorboard",
    logging_dir="./tensorboard/{}-finetuned-{}".format(model_name, task),
)

# Huggingface optimizers: https://huggingface.co/docs/transformers/v4.39.3/en/main_classes/optimizer_schedules#optimization
# Huggingface Schedulers: https://huggingface.co/docs/transformers/v4.39.3/en/main_classes/optimizer_schedules#schedules
# E.g.
# from transformers import AdamW, get_linear_schedule_with_warmup
# optimizer = AdamW(model.parameters(), lr=training_args.learning_rate)
# num_warmup_steps = training_args.num_train_epochs * len(train_dataset) // training_args.per_device_train_batch_size  # Adjust warmup based on your training setup
# scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=num_warmup_steps, num_training_steps=training_args.num_train_epochs * len(train_dataset) // training_args.per_device_train_batch_size)


# PyTorch optimizers: https://pytorch.org/docs/stable/optim.html#algorithms
# PyTorch schedulers: https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
optimizer = optim.SGD(model.parameters(), lr=training_args.learning_rate)
scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)


trainer = Trainer(
    model,
    args=training_args,
    optimizers=(optimizer, scheduler),
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    # compute_metrics=compute_metrics
)

In [ ]:
trainer.train()
# trainer.save_model("models/{}-finetuned-{}".format(model_name, task))

In [ ]:
# eval_result = trainer.evaluate()